In [1]:
import pymysql
import pandas as pd
import numpy as np
import datetime
import sys
sys.path.append(r'D:\jupyter files\waiting_time_project\my_tools')
import tools_for_os.for_df as ml_df

In [2]:
sql_appointment = """SELECT * FROM appointment"""
data_appointment = ml_df.get_df_from_sql(sql_appointment)
print(f'The shape of data_appointment {data_appointment.shape}')
sql_dx = """SELECT * FROM diagnosis"""
data_dx = ml_df.get_df_from_sql(sql_dx)
print(f'The shape of data_dx {data_dx.shape}')
sql_dxt = """SELECT * FROM diagnosistranslation"""
data_dxt = ml_df.get_df_from_sql(sql_dxt)
print(f'The shape of data_dxt {data_dxt.shape}')
sql_pt = """SELECT * FROM patient"""
data_pt = ml_df.get_df_from_sql(sql_pt)
print(f'The shape of data_pt {data_pt.shape}')
sql_co = """SELECT * FROM course"""
data_co = ml_df.get_df_from_sql(sql_co)
print(f'The shape of data_co {data_co.shape}')
sql_pl = """SELECT * FROM plan"""
data_pl = ml_df.get_df_from_sql(sql_pl)
print(f'The shape of data_pl {data_pl.shape}')
sql_pd = """SELECT * FROM patientdoctor"""
data_pd = ml_df.get_df_from_sql(sql_pd)
print(f'The shape of data_pd {data_pd.shape}')

The shape of data_appointment (1074478, 17)
The shape of data_dx (39486, 12)
The shape of data_dxt (1291, 6)
The shape of data_pt (51062, 2)
The shape of data_co (58360, 11)
The shape of data_pl (148421, 13)
The shape of data_pd (148770, 7)


In [3]:
try:
    data_appointment.drop(columns = ['LastUpdated'], inplace = True)
    data_dx.drop('LastUpdated', axis = 1, inplace = True)
    data_dxt.drop('LastUpdated', axis = 1, inplace = True)
    data_pt.drop('LastUpdated', axis = 1, inplace = True)
    data_co.drop('LastUpdated', axis = 1, inplace = True)
    data_pl.drop('LastUpdated', axis = 1, inplace = True)
    data_pd.drop('LastUpdated', axis = 1, inplace = True)
    print('Drop columns')
except:
    print('Finish droppping columns')

Drop columns


## Appointment 表

In [4]:
data_appointment.columns

Index(['AppointmentSerNum', 'PatientSerNum', 'AppointmentAriaSer',
       'ActivityInstanceAriaSer', 'DiagnosisSerNum', 'PrioritySerNum',
       'AliasSerNum', 'AliasExpressionSerNum', 'Status', 'State',
       'ScheduledStartTime', 'ScheduledEndTime', 'ActualStartDate',
       'ActualEndDate', 'EstimatedDuration', 'HstryUserName'],
      dtype='object')

In [5]:
data_appointment.ActualStartDate.iloc[0]

datetime.datetime(1970, 1, 1, 0, 0)

In [6]:
print(f'The shape of data_appointment {data_appointment.shape}')
data_appointment = data_appointment[
    (data_appointment.ActualStartDate != datetime.datetime(1970, 1, 1, 0, 0))&
    (data_appointment.ActualEndDate != datetime.datetime(1970, 1, 1, 0, 0))]
print(f'The shape of data_appointment {data_appointment.shape}')

The shape of data_appointment (1074478, 16)
The shape of data_appointment (370303, 16)


In [7]:
data_appointment.Status.value_counts()

Completed           369089
Pt. CompltFinish      1210
                         4
Name: Status, dtype: int64

In [8]:
data_appointment.State.value_counts()

Active     370296
                4
Deleted         3
Name: State, dtype: int64

In [9]:
print(f'The shape of data_appointment {data_appointment.shape}')
data_appointment = data_appointment[data_appointment.State == 'Active']
print(f'The shape of data_appointment {data_appointment.shape}')
data_appointment = data_appointment[(data_appointment.Status == 'Completed') | 
                                    (data_appointment.Status == 'Pt. CompltFinish')]
print(f'The shape of data_appointment {data_appointment.shape}')

The shape of data_appointment (370303, 16)
The shape of data_appointment (370296, 16)
The shape of data_appointment (370296, 16)


## PatientDoctor 表

In [10]:
data_pd.AliasSerNum.value_counts()

37    148770
Name: AliasSerNum, dtype: int64

##### PatientDoctor 表中，AliasSerNum 只有37

##### 筛选出 OncologistFlag = 1，PrimaryFlag = 1 的数据

In [11]:
print(f'The shape of data_pd {data_pd.shape}')
data_pd = data_pd[data_pd.OncologistFlag == 1]
print(f'The shape of data_pd {data_pd.shape}')
data_pd = data_pd[data_pd.PrimaryFlag == 1]
print(f'The shape of data_pd {data_pd.shape}')

The shape of data_pd (148770, 6)
The shape of data_pd (51535, 6)
The shape of data_pd (49874, 6)


## Plan 表

In [12]:
data_pl.head()

,PlanSerNum,CourseSerNum,PlanAriaSer,PlanId,AliasSerNum,AliasExpressionSerNum,TreatmentOrientation,CreationDate,Status,StatusDate,TreatmentType,Intent
0,1,40522,929513,FT1 RT BREAST,37,0,HFS,2015-08-03 09:53:00,TreatApproval,2015-08-05 11:56:00,Linac,CURATIVE
1,2,40438,929546,FT1 RT BREAST,37,0,HFS,2015-08-03 11:40:00,TreatApproval,2015-08-05 14:23:00,Linac,CURATIVE
2,3,40024,922090,FP1 RT BREAST,37,0,HFS,2015-06-25 16:15:00,TreatApproval,2015-06-29 14:52:00,Linac,CURATIVE
3,4,40569,928000,FT1 LT BREAST,37,0,HFS,2015-07-27 15:34:00,Retired,2015-08-04 09:16:00,Linac,CURATIVE
4,5,40644,929664,FT1 LT BREAST,37,0,HFS,2015-08-04 09:56:00,TreatApproval,2015-08-05 13:29:00,Linac,CURATIVE


In [13]:
data_pl.Status.value_counts()

Unapproved          74207
TreatApproval       47300
Rejected            14855
PlanApproval         4944
Retired              3873
CompletedEarly       2550
Completed             458
ExternalApproval      222
Reviewed               12
Name: Status, dtype: int64

In [14]:
data_pl.PlanId.value_counts()

EmergencyPS         7162
1                   1489
FP1 BRAIN           1395
ORIGINAL            1337
FINAL PLAN 1        1310
FP1 PROSTATE        1249
FILMS               1162
FP1 PELVIS          1155
BRAIN               1000
RT BREAST            972
LT BREAST            936
FP1 RBREAST          875
FILM                 835
FP1 ENT              829
2                    818
FP1 RT BREAST        789
FP1 LT BREAST        783
CF                   740
FP1 LBREAST          722
PROSTATE             653
PELVIS               626
FP1 RLUNG            619
FP1_BRAIN            588
FP2 LT BREAST        567
FP1 RT LUNG          522
FP1 RECTUM           512
FP2 RT BREAST        474
FP1 LLUNG            468
CFS                  466
PS1                  458
                    ... 
PL2 MOD                1
FinF13V                1
FP2_SPLEEN             1
KV/KV                  1
LB15 S2                1
NoWingHUOR ES          1
FP1 ORBITS:1           1
FP1_VERTEX             1
P1 LT CWorig:1         1


In [15]:
print(f'The shape of data_pl {data_pl.shape}')
data_pl_ = data_pl[data_pl.Status == 'TreatApproval']
print(f'The shape of data_pl_ {data_pl_.shape}')

The shape of data_pl (148421, 12)
The shape of data_pl_ (47300, 12)


In [16]:
data_pl_.PlanId.value_counts()

FP1 BRAIN        1093
FP1 PROSTATE     1074
FP1 RBREAST       768
FP1 PELVIS        727
FP1 RT BREAST     648
FP1 LBREAST       621
FP1 LT BREAST     615
FP2 LT BREAST     521
FP1 RLUNG         515
FP1_BRAIN         490
FP2 RT BREAST     436
FP2 LBREAST       412
FP1 RT LUNG       390
FP1 RECTUM        376
FP1 LLUNG         370
FP1 ENT           366
FP2 RBREAST       363
FP1 CHEST         327
FP2 PROSTATE      327
FP1 ABDOMEN       305
FP1 LT LUNG       276
FP1 WBI           263
FT1 LT BREAST     241
FP1 PELVIS #      240
FP1_PELVIS        238
FP1 LBREASTBH     230
C1_FP1_BRAIN      206
FT1 RT BREAST     205
FP1 BLADDER       204
FP2 RECTUM        200
                 ... 
FP1 P-AORTICS       1
ver FP2 BRAIN       1
FPL1T11-L1          1
TB3 1.8 FHI1        1
QA_EPID_6B:1        1
FP1_Lpelvis         1
FPL1 LTCW           1
FPL1-LTC/WALL       1
FP1_LT ORBIT        1
FP1_mc              1
QA_FPM1A            1
QA_IMAGING1         1
FP1 ENT SUP         1
FP1 Scalp RT        1
RT 9TH RIB

In [17]:
##### 

##### ============按照之前的方法筛数据===================

##### Appointment 的预处理

In [18]:
print(f'The shape of data_appointment {data_appointment.shape}')
data_appointment_ = data_appointment[(data_appointment.AliasSerNum == 31) &
                                     (data_appointment.AliasSerNum == 23) &
                                     (data_appointment.ActualStartDate != datetime.datetime(1970, 1, 1, 0, 0)) &
                                     (data_appointment.ActualEndDate != datetime.datetime(1970, 1, 1, 0, 0)) &
                                     (data_appointment.ActualStartDate != data_appointment.ActualEndDate)]
print(f'The shape of data_appointment_ {data_appointment_.shape}')

data_appointment_ = data_appointment_[(data_appointment_.State == 'Active') &
                                     ((data_appointment_.Status == 'Completed') |
                                     (data_appointment_.Status == 'Pt. CompltFinish'))]
print(f'The shape of data_appointment_ {data_appointment_.shape}')

The shape of data_appointment (370296, 16)
The shape of data_appointment_ (0, 16)
The shape of data_appointment_ (0, 16)


##### PatientDoctor 的预处理

In [23]:
print(f'The shape of data_pd {data_pd.shape}')
data_pd_ = data_pd[(data_pd.OncologistFlag == 1) &
                   (data_pd.PrimaryFlag == 1)]
print(f'The shape of data_pd_ {data_pd_.shape}')

The shape of data_pd (49874, 6)
The shape of data_pd_ (49874, 6)


##### Plan 的预处理

In [25]:
print(f'The shape of data_pl {data_pl.shape}')
data_pl_ = data_pl[data_pl.Status == 'TreatApproval']
data_pl_.TreatmentOrientation = data_pl_.TreatmentOrientation.apply(
    lambda x: 'NULL' if x == '' else x)
print(f'The shape of data_pl_ {data_pl_.shape}')

The shape of data_pl (148421, 12)
The shape of data_pl_ (47300, 12)


C:\Users\WENDY\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


##### Course 的预处理

In [27]:
data_co.CourseId.value_counts()

C1                  24054
1                   10772
C2                   5113
C3                   1997
2                    1415
C4                    903
BRACHY                569
PHYSICSQA_C           552
MD ORIG_C1            498
PHYSICS QA            453
C5                    426
RADCALC_C1            418
3                     378
MD ORIG_C2            274
Brachy                268
C6                    213
TomoDQA_C1            213
C1_CK_DOSE            174
PHYSICS_QA            164
PhysicsQA_C           149
hypo_study            144
NOT USED              135
MD ORIG_C3            128
4                     125
Physics QA            120
C7                    114
PHYSICS_QA_C1         102
TEACHING              102
MIM SUM C1+C2          99
PHYSICSQA_C1           93
                    ...  
newdose                 1
CONTOUR SIMPLE          1
NOTUSED2                1
TB3 RARC SE             1
Ignore-RR T             1
test_epoon              1
Oblique Pai             1
RADCALC_C8  

In [20]:
data_pl.TreatmentOrientation.value_counts()

HFS     102510
NULL     41850
FFS       3157
HFP        605
FFP        186
HFDR        41
HFDL        36
FFDL        28
FFDR         8
Name: TreatmentOrientation, dtype: int64

The shape of data_pd (49874, 6)


In [22]:
data_appointment.AliasSerNum.iloc[0]

2

##### ====================================================

## 合并数据